## 🍸 칵테일 AI 검색 시스템
### Task
- **C1**: 시각적 유사성 기반 칵테일 추천
- **C2**: 맛 프로파일 분석 (0-5 스케일)
- **C3**: 칵테일 분류 및 계통 정보
- **C4**: 레시피 및 재료 치환 정보

In [1]:
# 메모리 정리 함수
import gc

def cleanup_memory():
    """메모리 정리 및 가비지 컬렉션"""
    # 기존 retrieval_system이 있다면 닫기
    if 'retrieval_system' in globals():
        try:
            retrieval_system.close()
            del globals()['retrieval_system']
            print("✅ 기존 검색 시스템 연결 해제")
        except:
            pass
    
    # 가비지 컬렉션 수행
    collected = gc.collect()
    print(f"🧹 가비지 컬렉션 완료: {collected}개 객체 정리")
    
    # 캐시 정리 옵션
    try:
        import os
        cache_file = "embedding_cache.json"
        if os.path.exists(cache_file):
            size_mb = os.path.getsize(cache_file) / 1024 / 1024
            print(f"📦 임베딩 캐시 파일 크기: {size_mb:.2f} MB")
            # 캐시가 너무 크면 경고
            if size_mb > 100:
                print("⚠️ 캐시 파일이 매우 큽니다. 필요시 수동으로 삭제하세요.")
    except:
        pass
    
    print("✨ 메모리 정리 완료!")

# 메모리 정리 실행
cleanup_memory()

🧹 가비지 컬렉션 완료: 0개 객체 정리
📦 임베딩 캐시 파일 크기: 48.97 MB
✨ 메모리 정리 완료!


## 🧹 메모리 정리 (메모리 사용량이 높을 때 실행)

In [2]:
# 필요한 라이브러리 import
import os
import json
import openai
from neo4j import GraphDatabase
from dotenv import load_dotenv
from typing import List, Dict
import re
from retrieval import CocktailRetrieval
from IPython.display import display, Markdown, HTML
import warnings
warnings.filterwarnings('ignore')

print("✅ 모든 라이브러리가 성공적으로 로드되었습니다!")

✅ 모든 라이브러리가 성공적으로 로드되었습니다!


In [ ]:
# 시스템 초기화
try:
    retrieval_system = CocktailRetrieval()
    print("🔥 칵테일 검색 시스템이 초기화되었습니다!")
    print("📊 Neo4j 데이터베이스 연결 완료")
    print("🤖 OpenAI GPT 연결 완료")
    print("📝 config.json 기반 프롬프트 시스템 로드 완료")
    print("⚙️ 임베딩 설정:", retrieval_system.prompt_loader.config.get('embedding_model', 'text-embedding-3-small'))
except Exception as e:
    print(f"❌ 시스템 초기화 실패: {e}")
    print("환경 변수(.env 파일)와 config.json을 확인해주세요.")

🔥 칵테일 검색 시스템이 초기화되었습니다!
📊 Neo4j 데이터베이스 연결 완료
🤖 OpenAI GPT 연결 완료
📝 config.json 기반 프롬프트 시스템 로드 완료
⚙️ 임베딩 설정: text-embedding-3-small


In [ ]:
def classify_task_automatically(question: str) -> tuple:
    """
    LLM을 사용하여 질문을 분석하고 적절한 태스크 카테고리와 타입을 결정
    """
    try:
        # config.json 기반 분류 정보 로드
        classifier_config = retrieval_system.prompt_loader.get_task_classifier_prompt()
        
        # 프롬프트와 설정 추출
        system_prompt = classifier_config.get('system_prompt', 'You are a task classifier.')
        user_prompt_template = classifier_config.get('user_prompt', '')
        temperature = classifier_config.get('temperature', 0.1)
        max_tokens = classifier_config.get('max_tokens', 200)
        model = classifier_config.get('model', 'gpt-4o-mini')
        
        # 사용자 질문을 프롬프트에 삽입
        user_message = f"{user_prompt_template}\n\n**사용자 질문:** \"{question}\""
        
        response = openai.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_message}
            ],
            temperature=temperature,
            max_tokens=max_tokens
        )
        
        # JSON 응답 파싱
        result = json.loads(response.choices[0].message.content)
        
        best_task = result['task']
        confidence = result['confidence']
        reason = result['reason']
        
        # 점수 생성 (선택된 태스크는 confidence 점수, 나머지는 낮은 점수)
        base_score = max(0, 100 - confidence) // 3
        scores = {
            'C1': base_score,
            'C2': base_score, 
            'C3': base_score,
            'C4': base_score
        }
        scores[best_task] = confidence
        
        print(f"🤖 config.json 기반 LLM 태스크 분류 완료")
        print(f"📋 선택된 태스크: {best_task} (신뢰도: {confidence}%)")
        print(f"💭 판단 근거: {reason}")
        
        # 태스크 타입 결정
        task_types = {
            'C1': 'visual_recommendation',
            'C2': 'taste_analysis', 
            'C3': 'classification_info',
            'C4': 'recipe_ingredients'
        }
        
        return best_task, task_types[best_task], scores
        
    except Exception as e:
        print(f"❌ LLM 기반 분류 실패: {e}")
        raise Exception(f"태스크 분류 중 오류가 발생했습니다: {str(e)}")

print("🤖 config.json 기반 LLM 태스크 분류 함수가 준비되었습니다!")

🤖 config.json 기반 LLM 태스크 분류 함수가 준비되었습니다!


In [ ]:
def display_results(question: str, context: Dict, llm_response: str, task_category: str, task_type: str, scores: Dict):
    """
    결과를 예쁘게 포맷팅하여 출력
    """
    # 태스크 정보 표시
    task_names = {
        'C1': '🎨 시각 인식 & 유사 추천',
        'C2': '👅 테이스팅 & 맛 프로파일',
        'C3': '📚 분류 & 계통 정보',
        'C4': '🍹 레시피 & 재료 정보'
    }
    
    # 검색 전략 정보 업데이트
    strategy_info = {
        'C1': '🔍 imageDescription + instructions 임베딩 혼합',
        'C2': '🔍 description + instructions 임베딩 혼합', 
        'C3': '🔍 description + 그래프 관계 중심',
        'C4': '🔍 instructions + measure 속성 활용'
    }
    
    # 검색 결과를 먼저 표시
    if 'cocktails' in context:
        display(Markdown("## 🔍 검색된 칵테일들"))
        for i, cocktail in enumerate(context['cocktails'], 1):
            similarity_score = cocktail.get('similarity_score', 'N/A')
            relevance_score = cocktail.get('relevance_score', 'N/A')
            
            cocktail_info = f"""
### {i}. {cocktail['name']} 
**유사도 점수:** {similarity_score if similarity_score != 'N/A' else relevance_score}

**설명:** {cocktail.get('description', 'N/A')}

**재료:** {', '.join(cocktail.get('ingredients', []))}

**카테고리:** {cocktail.get('category', 'N/A')} | **글라스:** {cocktail.get('glass_type', 'N/A')}
"""
            
            if 'recipe_with_measures' in cocktail:
                recipe_items = [f"{item['ingredient']} ({item['measure']})" for item in cocktail['recipe_with_measures']]
                cocktail_info += f"\n**레시피:** {', '.join(recipe_items)}"
            
            display(Markdown(cocktail_info))
            
    elif 'results' in context:
        display(Markdown("## 🔍 검색 결과"))
        for i, result in enumerate(context['results'], 1):
            if 'ingredient' in result:
                display(Markdown(f"**{i}. {result['ingredient']}** (사용빈도: {result['usage_frequency']}, 유사도: {result.get('similarity', 'N/A')})"))
            elif 'category_name' in result:
                display(Markdown(f"**{i}. {result['category_name']}** (패밀리 크기: {result['family_size']})"))
            elif 'name' in result:
                display(Markdown(f"**{i}. {result['name']}**"))
                if 'detailed_recipe_with_measures' in result:
                    recipe_items = [f"{item['ingredient']} ({item['measure']})" for item in result['detailed_recipe_with_measures']]
                    display(Markdown(f"레시피: {', '.join(recipe_items)}"))
    
    # AI 분석 결과 표시
    display(HTML(f"""
    <div style="background-color: #f0f8ff; padding: 20px; border-radius: 10px; margin: 10px 0;">
        <h2 style="color: #2e86ab; margin-top: 0;">🤖 AI 분석 결과</h2>
        <p><strong>📝 질문:</strong> {question}</p>
        <p><strong>🎯 감지된 태스크:</strong> {task_names[task_category]}</p>
        <p><strong>🔍 검색 전략:</strong> {strategy_info.get(task_category, context.get('embedding_strategy', 'N/A'))}</p>
        <p><strong>📊 태스크 점수:</strong> C1:{scores['C1']} | C2:{scores['C2']} | C3:{scores['C3']} | C4:{scores['C4']}</p>
    </div>
    """))
    
    # LLM 답변 표시
    display(HTML(f"""
    <div style="background-color: #fff8e1; padding: 20px; border-radius: 10px; margin: 20px 0; border-left: 5px solid #ffa726;">
        <h2 style="color: #f57c00; margin-top: 0;">🤖 AI 전문가 답변</h2>
        <div style="white-space: pre-wrap; font-family: Arial, sans-serif; line-height: 1.6;">{llm_response}</div>
    </div>
    """))

print("🎨 결과 표시 함수가 준비되었습니다!")

🎨 결과 표시 함수가 준비되었습니다!


## 🚀 칵테일 질문
### 예시 질문들:
- `"빨간색이고 체리가 올라간 칵테일과 비슷한 칵테일 추천해줘"`
- `"네그로니의 맛 프로파일을 달/신/쌉/짠/감칠 0-5 스케일로 알려줘"`
- `"다이키리의 분류 정보를 알려줘 - 패밀리, 베이스 스피릿, 스타일, 글라스"`
- `"맨해튼 레시피를 정확한 비율과 측정값으로 알려줘"`

In [ ]:
# C1 테스크 : 시각 유사성 기반 칵테일 추천
question = "빨간색이고 체리가 올라간 칵테일과 비슷한 칵테일 추천해줘"

# ========== 이 아래는 수정하지 마세요! ==========

try:
    # 자동 태스크 분류
    task_category, task_type, scores = classify_task_automatically(question)
    
    print(f"🤖 질문 분석 중...")
    print(f"🎯 감지된 태스크: {task_category} ({task_type})")
    print(f"🔍 검색 시작...")
    
    # 검색 및 답변 생성 (출력 억제)
    import sys
    from io import StringIO
    
    # 기존 stdout 저장
    old_stdout = sys.stdout
    sys.stdout = StringIO()
    
    try:
        # 검색 실행 (출력 없이)
        if task_category == "C1":
            context = retrieval_system.c1_visual_similarity(question, task_type)
        elif task_category == "C2":
            context = retrieval_system.c2_taste_profile(question, task_type)
        elif task_category == "C3":
            context = retrieval_system.c3_classification(question, task_type)
        elif task_category == "C4":
            context = retrieval_system.c4_recipe_ingredients(question, task_type)
        else:
            raise ValueError(f"알 수 없는 태스크 카테고리: {task_category}")
        
        # LLM 답변 생성 (출력 없이)
        llm_response = retrieval_system.query_llm(context, question, task_category)
        
    finally:
        # stdout 복원
        sys.stdout = old_stdout
    
    print(f"✅ 검색 완료! 결과를 표시합니다...")
    
    # 결과 표시
    display_results(question, context, llm_response, task_category, task_type, scores)
    
except Exception as e:
    display(HTML(f"""
    <div style="background-color: #ffebee; padding: 20px; border-radius: 10px; margin: 10px 0; border-left: 5px solid #f44336;">
        <h3 style="color: #d32f2f; margin-top: 0;">❌ 오류 발생</h3>
        <p><strong>오류 내용:</strong> {str(e)}</p>
        <p><strong>해결 방법:</strong></p>
        <ul>
            <li>네트워크 연결 상태를 확인해주세요</li>
            <li>OpenAI API 키가 유효한지 확인해주세요</li>
            <li>Neo4j 데이터베이스 연결 상태를 확인해주세요</li>
            <li>질문을 다르게 표현해보세요</li>
        </ul>
    </div>
    """))

🤖 config.json 기반 LLM 태스크 분류 완료
📋 선택된 태스크: C1 (신뢰도: 85%)
💭 판단 근거: 색상과 외관을 기반으로 비슷한 칵테일 추천을 요청하는 질문이므로 C1이 적합
🤖 질문 분석 중...
🎯 감지된 태스크: C1 (visual_recommendation)
🔍 검색 시작...
✅ 검색 완료! 결과를 표시합니다...


## 🔍 검색된 칵테일들

In [ ]:
# C2 테스트: 맛 프로파일 분석
c2_question = "네그로니의 맛 프로파일을 달콤함/신맛/쓴맛/짠맛/감칠맛 0-5 스케일로 알려줘"

try:
    # 자동 태스크 분류
    task_category, task_type, scores = classify_task_automatically(c2_question)
    
    print(f"🤖 C2 테스트 - 질문 분석 중...")
    print(f"🎯 감지된 태스크: {task_category} ({task_type})")
    print(f"🔍 검색 시작...")
    
    # 검색 및 답변 생성 (출력 억제)
    old_stdout = sys.stdout
    sys.stdout = StringIO()
    
    try:
        context = retrieval_system.c2_taste_profile(c2_question, task_type)
        llm_response = retrieval_system.query_llm(context, c2_question, task_category)
    finally:
        sys.stdout = old_stdout
    
    print(f"✅ C2 검색 완료! 결과를 표시합니다...")
    display_results(c2_question, context, llm_response, task_category, task_type, scores)
    
except Exception as e:
    display(HTML(f"""
    <div style="background-color: #ffebee; padding: 20px; border-radius: 10px; margin: 10px 0; border-left: 5px solid #f44336;">
        <h3 style="color: #d32f2f; margin-top: 0;">❌ C2 테스트 오류</h3>
        <p><strong>오류 내용:</strong> {str(e)}</p>
    </div>
    """))

🤖 config.json 기반 LLM 태스크 분류 완료
📋 선택된 태스크: C2 (신뢰도: 85%)
💭 판단 근거: 맛 프로파일 분석을 요청하는 질문이므로 C2가 적합
🤖 C2 테스트 - 질문 분석 중...
🎯 감지된 태스크: C2 (taste_analysis)
🔍 검색 시작...
✅ C2 검색 완료! 결과를 표시합니다...


## 🔍 검색된 칵테일들

In [ ]:
# C3 테스트: 분류 및 계통 정보
c3_question = "다이키리의 분류 정보를 알려줘 - 패밀리, 베이스 스피릿, 스타일, 글라스"

try:
    # 자동 태스크 분류
    task_category, task_type, scores = classify_task_automatically(c3_question)
    
    print(f"🤖 C3 테스트 - 질문 분석 중...")
    print(f"🎯 감지된 태스크: {task_category} ({task_type})")
    print(f"🔍 검색 시작...")
    
    # 검색 및 답변 생성 (출력 억제)
    old_stdout = sys.stdout
    sys.stdout = StringIO()
    
    try:
        context = retrieval_system.c3_classification(c3_question, task_type)
        llm_response = retrieval_system.query_llm(context, c3_question, task_category)
    finally:
        sys.stdout = old_stdout
    
    print(f"✅ C3 검색 완료! 결과를 표시합니다...")
    display_results(c3_question, context, llm_response, task_category, task_type, scores)
    
except Exception as e:
    display(HTML(f"""
    <div style="background-color: #ffebee; padding: 20px; border-radius: 10px; margin: 10px 0; border-left: 5px solid #f44336;">
        <h3 style="color: #d32f2f; margin-top: 0;">❌ C3 테스트 오류</h3>
        <p><strong>오류 내용:</strong> {str(e)}</p>
    </div>
    """))

🤖 config.json 기반 LLM 태스크 분류 완료
📋 선택된 태스크: C3 (신뢰도: 90%)
💭 판단 근거: 칵테일의 분류, 패밀리, 베이스 스피릿, 제조 스타일 및 글라스웨어에 대한 정보를 요청하고 있으므로 C3가 적합
🤖 C3 테스트 - 질문 분석 중...
🎯 감지된 태스크: C3 (classification_info)
🔍 검색 시작...
✅ C3 검색 완료! 결과를 표시합니다...


## 🔍 검색 결과

In [ ]:
# C4 테스트: 레시피 및 재료 정보
c4_question = "맨해튼 레시피를 정확한 비율과 측정값으로 알려줘"

try:
    # 자동 태스크 분류
    task_category, task_type, scores = classify_task_automatically(c4_question)
    
    print(f"🤖 C4 테스트 - 질문 분석 중...")
    print(f"🎯 감지된 태스크: {task_category} ({task_type})")
    print(f"🔍 검색 시작...")
    
    # 검색 및 답변 생성 (출력 억제)
    old_stdout = sys.stdout
    sys.stdout = StringIO()
    
    try:
        context = retrieval_system.c4_recipe_ingredients(c4_question, task_type)
        llm_response = retrieval_system.query_llm(context, c4_question, task_category)
    finally:
        sys.stdout = old_stdout
    
    print(f"✅ C4 검색 완료! 결과를 표시합니다...")
    display_results(c4_question, context, llm_response, task_category, task_type, scores)
    
except Exception as e:
    display(HTML(f"""
    <div style="background-color: #ffebee; padding: 20px; border-radius: 10px; margin: 10px 0; border-left: 5px solid #f44336;">
        <h3 style="color: #d32f2f; margin-top: 0;">❌ C4 테스트 오류</h3>
        <p><strong>오류 내용:</strong> {str(e)}</p>
    </div>
    """))

🤖 config.json 기반 LLM 태스크 분류 완료
📋 선택된 태스크: C4 (신뢰도: 90%)
💭 판단 근거: 정확한 레시피와 측정값을 요청하는 질문이므로 C4가 적합
🤖 C4 테스트 - 질문 분석 중...
🎯 감지된 태스크: C4 (recipe_ingredients)
🔍 검색 시작...
✅ C4 검색 완료! 결과를 표시합니다...


## 🔍 검색 결과

In [ ]:
# 🛠️ 시스템 정리 (노트북 사용 완료 시 실행)
try:
    if 'retrieval_system' in globals():
        retrieval_system.close()
        print("🔄 시스템 연결이 정리되었습니다.")
        print("💾 임베딩 캐시가 저장되었습니다.")
    else:
        print("⚠️ 시스템이 초기화되지 않았습니다.")
except Exception as e:
    print(f"❌ 시스템 정리 중 오류: {e}")

🔄 시스템 연결이 정리되었습니다.
💾 임베딩 캐시가 저장되었습니다.
